In [2]:
%matplotlib inline
import matplotlib as mpl
#mpl.use("pgf")

import itertools
from skimage import io
import numpy as np
import glob
import math
import matplotlib
from matplotlib.ticker import FuncFormatter, MultipleLocator
import matplotlib.pyplot as plt
# import brewer2mpl
from pylab import rcParams
import pandas
from itertools import tee

import seaborn as sns

In [3]:
white_background = True
cite_latex = False
cite_thesis = False

In [4]:
sns.set_context("paper", font_scale=1.0)
if white_background:
    sns.set(color_codes=True, style="whitegrid")
    font_color_label = "black"
else:
    sns.set(color_codes=True, style="ticks")
    plt.style.use("dark_background")
    font_color_label = "white"

In [5]:
def figsize(scale, inverted=False):
    fig_width_pt = 237.2                            # Get this from LaTeX using \the\textwidth or \the\linewidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width/golden_mean if inverted else fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

In [6]:
pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "sans-serif",
    "font.serif": ['Computer Modern Roman'],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 8,               # LaTeX default is 10pt font.
    "font.size": 9,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "axes.titlesize": 9,
    "pdf.fonttype": 42,
    "pdf.use14corefonts": True,
    "ps.useafm": True,
    "svg.fonttype": 'none',
    "figure.figsize": [2.8, 1.85],     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}", # plots will be generated using this preamble
        r"\usepackage{xcolor}",
        r"\usepackage{adjustbox}",
        r"\newcommand{\citetalias}[1]{\cite{#1}}"
        r"\definecolor{mycolor0}{rgb}{0.12156862745098039, 0.47058823529411764, 0.7058823529411765}",
        r"\definecolor{mycolor2}{rgb}{0.2, 0.6274509803921569, 0.17254901960784313}",
        r"\definecolor{mycolor4}{rgb}{0.8901960784313725, 0.10196078431372549, 0.10980392156862745}",
        r"\definecolor{mycolor6}{rgb}{1.0, 0.4980392156862745, 0.0}",
        r"\definecolor{mycolor8}{rgb}{0.41568627450980394, 0.23921568627450981, 0.6039215686274509}",
        ]
}
rcParams.update(pgf_with_latex)

In [7]:
# scores_list_proposed_mpii = [2.5519632972299306,
#  3.2762327739027439,
#  4.7310029860358647,
#  4.6895662220437355,
#  3.2641634944056581,
#  3.9574127379659494,
#  3.4303666265798229,
#  4.1917410052093249,
#  6.8629787764084726,
#  4.1668642443833619,
#  3.7205894341632555,
#  4.2139113055287147,
#  4.2053779655434491,
#  4.2392865293390942,
#  2.8633420743831333]
# mean_proposed_mpii = np.mean(scores_list_proposed_mpii)
# std_proposed_mpii = np.std(scores_list_proposed_mpii)
# print('Mean', mean_proposed_mpii, 'Std', std_proposed_mpii)

In [8]:
# methods = ['MPII', 'Proposed']
# performance = [mean_mpii, mean_proposed]
# performance_error = [std_mpii, std_proposed]

# methods = ['Single eye~\cite{Zhang2015e}', 'iTracker~\cite{Krafka2016}', 'iTracker (Two eyes)', 'iTracker (AlexNet)', 'Single face', 'Spatial weights CNN~\cite{zhang17cvprw}', 'Proposed']
# performance = [6.7, 6.2, 6.2, 5.6, 5.5, 4.8, mean_proposed_mpii]
# performance_error = [1.4, 1.0, 0.9, 0.7, 0.9, 0.8, std_proposed_mpii]

if cite_latex:
    methods = [r'Single eye~\cite{Zhang2015e}', 
               r'iTracker~\cite{Krafka2016}', 
               r'iTracker (AlexNet)~\cite{Krafka2016, zhang17cvprw}', 
               r'Spatial weights CNN~\cite{zhang17cvprw}', 
               r'Spatial weights CNN (ensemble)', 
               r'Proposed: 1 model', 
               r'Proposed: 2 model ensemble', 
               r'\textbf{Proposed: 4 model ensemble}']
else:
    methods = ['Single eye CVPR2015', 
               'iTracker CVPR2016', 
               'iTracker (AlexNet) CVPR2016; CVPR2017W', 
               'Spatial weights CNN CVPR2017W', 
               'Spatial weights CNN (ensemble)', 
               'Proposed: 1 model', 
               'Proposed: 2 model ensemble', 
               r'\textbf{Proposed: 4 model ensemble}']

if cite_thesis:
    methods_alias = []
    for method in methods:
        methods_alias.append(method.replace('cite', 'citetalias'))
    methods = methods_alias

performance = [6.7, 6.2, 5.6, 4.8, 4.8, 4.8, 4.6, 4.3]
performance_error = [1.4, 1.0, 0.7, 0.8, 0.8, 1.0, 1.2, 0.9]

In [9]:
y_pos = [0.0, 1.0, 2.0, 3.0, 4.0, 5.35, 6.35, 7.35]

In [10]:
palette = sns.color_palette("Reds", 6)[0:5] + sns.color_palette("Blues", 4)[1:4]
rects = plt.bar(y_pos, performance, 0.85, align='center', alpha=0, yerr=performance_error, 
                error_kw={'elinewidth': 1, 'capsize': 3, 'capthick': 1, 'ecolor': 'xkcd:steel grey'})

patterns = ['\\\\', 'xxx', '++', '--', '-', '|||', '//', '*']
patterns_legend = ['\\\\\\', 'xxx', '+++', '--', '----', '|||', '///', '**']

method_idx = [0, 1, 2, 3, 4, 5, 6, 7]

for idx, (rect, label) in enumerate(zip(rects, performance)):
    label_print = r'{\fontseries{sbc}\selectfont %.1f}' % label
    left, bottom = rect.get_xy()
    pattern = patterns[method_idx[idx]]
    # rect.set_hatch(pattern)
    color = palette[method_idx[idx]]
    plt.fill_between([left, left+rect.get_width()], rect.get_height(), 1, hatch=pattern, edgecolor=font_color_label, facecolor=color)
    plt.fill_between([left, left+rect.get_width()], 1, 0, hatch=pattern, edgecolor=font_color_label, facecolor=color, alpha=0.25)
    plt.text(rect.get_x() + rect.get_width()/2, 0.15, label_print, ha='center', va='bottom', color=font_color_label)

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off') # labels along the bottom edge are off
plt.ylabel('3D Angular Error (degrees)')
l = plt.legend(rects, methods, loc='upper center', bbox_to_anchor=(0.395, 2.05),
          ncol=1, fancybox=True, shadow=True)

plt.plot([(5.35-4.0)/2+4.0, (5.35-4.0)/2+4.0], [0, 9], '--', linewidth=1, color='lightgray')
plt.ylim(0, 9)

for idx, patch in enumerate(l.get_patches()):
    patch.set_alpha(1)
    patch.set_color(palette[method_idx[idx]])
    patch.set_edgecolor(font_color_label)
    patch.set_hatch(patterns_legend[method_idx[idx]])

# plt.axes().yaxis.set_minor_locator(MultipleLocator(1))
# plt.gca().yaxis.grid(True, which='minor', color='lightgray')
plt.gca().xaxis.grid(False)
plt.savefig('dataset_statistics/accuracy_mpii/accuracy_mpii_gaze' + ('_' if cite_latex else '_nocite_') + ('white' if white_background else 'black') + '.pdf', bbox_inches='tight')
plt.savefig('dataset_statistics/accuracy_mpii/accuracy_mpii_gaze' + ('_' if cite_latex else '_nocite_') + ('white' if white_background else 'black') + '.pgf', bbox_inches='tight')

plt.show()
plt.close()

/home/human/tensorflow/local/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


OSError: [Errno 2] No such file or directory: 'latex'

/home/human/tensorflow/local/lib/python2.7/site-packages/matplotlib/font_manager.py:1328: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


OSError: [Errno 2] No such file or directory: 'latex'

<Figure size 201.6x133.2 with 1 Axes>